In [ ]:
import pandas as pd
df = pd.read_csv('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz', compression='gzip')
df.columns = ['dt'] + df.columns[1:].tolist() # 更改欄位名稱
test_data = pd.read_csv('..\data\需預測的顧客名單及提交檔案範例.csv')

In [3]:
train_data = df
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

(32975654, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [19]:
feature_columns = list(set(train_data.columns.tolist()) - set(['chid','shop_tag']))
categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
numerical_features = list(set(feature_columns)-set(categorical_features))
train_df = train_data
sample_submit = test_data
len(feature_columns),len(categorical_features),len(numerical_features)

(51, 9, 42)

In [20]:
from tqdm import tqdm
for idx in tqdm(sample_submit.index[:3]): # demo用所以取前五筆 實際上應該取全部
    # 取出這一條row的chid
    chid = sample_submit.loc[idx,'chid']
    
    # 從歷史訓練數據抓對應的chid 算一些統計資訊 當作特徵 併入sample_submit裡面

    # 數值特徵統計資訊
    for col in tqdm(numerical_features):
        # 1.平均值  2.最大值  3.最小值 
        sample_submit.loc[idx,f"mean_of_{col}"] = train_df.loc[train_df['chid']==chid,col].mean()
        sample_submit.loc[idx,f"max_of_{col}"] = train_df.loc[train_df['chid']==chid,col].max()
        sample_submit.loc[idx,f"min_of_{col}"] = train_df.loc[train_df['chid']==chid,col].min()
        
    # 類別特徵統計資訊
    for col in tqdm(categorical_features):
        # 1.nunique (這個欄位有多少種不一樣的數值)
        sample_submit.loc[idx,f"nunique_of_{col}"] = train_df.loc[train_df['chid']==chid,col].nunique()
        
        # 2.norm_count (這個欄位在每一種數值上出現的次數(count)除以總數(normalize)等於取平均(mean))
        cat_feat = pd.get_dummies(train_df[col],columns=col) # one hot 格式
        cat_feat = pd.DataFrame(cat_feat.mean(axis=0)).T # 取平均
        cat_feat.index = [idx] # 處理DataFrame格式
        cat_feat.columns = [ "norm_count_"+col+"_"+str(i) for i in cat_feat.columns] # 處理DataFrame格式
        print(sample_submit.shape,cat_feat.shape)
        display(cat_feat)
        try:
            sample_submit = sample_submit.join(cat_feat)# 合併DataFrame
        except:
            sample_submit.loc[idx,cat_feat.columns] = cat_feat.values.reshape(-1)
        print(sample_submit.shape,cat_feat.shape)

sample_submit.head(3)

100%|██████████| 42/42 [00:23<00:00,  1.76it/s]


(500000, 131) (1, 3)


,norm_count_masts_1.0,norm_count_masts_2.0,norm_count_masts_3.0
0,0.475057,0.524312,0.00063


(500000, 134) (1, 3)
(500000, 135) (1, 6)


,norm_count_educd_1.0,norm_count_educd_2.0,norm_count_educd_3.0,norm_count_educd_4.0,norm_count_educd_5.0,norm_count_educd_6.0
0,0.011728,0.139897,0.432235,0.16215,0.151359,0.102631


...

(500000, 237) (1, 9)
(500000, 237) (1, 2)


,norm_count_primary_card_0.0,norm_count_primary_card_1.0
2,0.083689,0.916311


100%|██████████| 3/3 [01:30<00:00, 30.26s/it]

(500000, 237) (1, 2)


,chid,top1,top2,top3,mean_of_card_9_txn_cnt,max_of_card_9_txn_cnt,min_of_card_9_txn_cnt,mean_of_card_14_txn_cnt,max_of_card_14_txn_cnt,min_of_card_14_txn_cnt,...,norm_count_age_3.0,norm_count_age_4.0,norm_count_age_5.0,norm_count_age_6.0,norm_count_age_7.0,norm_count_age_8.0,norm_count_age_9.0,nunique_of_primary_card,norm_count_primary_card_0.0,norm_count_primary_card_1.0
0,10128239,18,10,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311
1,10077943,48,22,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311
2,10277876,22,10,39,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311


In [21]:
sample_submit.head(10)

,chid,top1,top2,top3,mean_of_card_9_txn_cnt,max_of_card_9_txn_cnt,min_of_card_9_txn_cnt,mean_of_card_14_txn_cnt,max_of_card_14_txn_cnt,min_of_card_14_txn_cnt,...,norm_count_age_3.0,norm_count_age_4.0,norm_count_age_5.0,norm_count_age_6.0,norm_count_age_7.0,norm_count_age_8.0,norm_count_age_9.0,nunique_of_primary_card,norm_count_primary_card_0.0,norm_count_primary_card_1.0
0,10128239,18,10,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311
1,10077943,48,22,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311
2,10277876,22,10,39,0.0,0.0,0.0,0.0,0.0,0.0,...,0.330446,0.274418,0.156851,0.058793,0.009312,0.000748,0.000024,1.0,0.083689,0.916311
3,10364842,37,19,13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10392717,2,36,26,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10439550,22,26,18,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10285755,12,13,36,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10335334,26,18,48,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10072243,15,13,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10274006,22,6,39,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
pd.options.display.max_rows = 999
print(sample_submit.shape)
sample_submit.describe().T

(500000, 237)


,count,mean,std,min,25%,50%,75%,max
chid,500000.0,1.025000e+07,1.443377e+05,1.000000e+07,1.012500e+07,1.025000e+07,1.037500e+07,1.050000e+07
top1,500000.0,2.180537e+01,1.241736e+01,2.000000e+00,1.200000e+01,1.900000e+01,3.600000e+01,4.800000e+01
top2,500000.0,2.182171e+01,1.241803e+01,2.000000e+00,1.300000e+01,1.900000e+01,3.600000e+01,4.800000e+01
top3,500000.0,2.181598e+01,1.240767e+01,2.000000e+00,1.300000e+01,2.100000e+01,3.600000e+01,4.800000e+01
mean_of_card_9_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max_of_card_9_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
min_of_card_9_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
mean_of_card_14_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max_of_card_14_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
min_of_card_14_txn_cnt,3.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [25]:
a = pd.DataFrame()
a['features_name'] = sample_submit.columns.tolist()
a

,features_name
0,chid
1,top1
2,top2
3,top3
4,mean_of_card_9_txn_cnt
5,max_of_card_9_txn_cnt
6,min_of_card_9_txn_cnt
7,mean_of_card_14_txn_cnt
8,max_of_card_14_txn_cnt
9,min_of_card_14_txn_cnt


# 保存

In [26]:
#train_data.reset_index().to_feather('../data/train_data.feather')
#test_data.reset_index().to_feather('../data/test_data.feather')